In [ ]:
# Install required packages
%pip install torch torchvision matplotlib pandas seaborn numpy


In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


In [ ]:
# Clone repository from GitHub
!git clone https://github.com/Mahad811/GenAi.git
%cd GenAi/Q3


In [ ]:
# Download CIFAR-10 dataset if not available
import urllib.request
import os

if not os.path.exists('cifar-10-python.tar.gz'):
    print("Downloading CIFAR-10 dataset...")
    urllib.request.urlretrieve(
        'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
        'cifar-10-python.tar.gz'
    )
    print("Download complete!")


In [ ]:
# Train PixelCNN (Uniform reduced configuration)
!python -m src.train \
    --model_type pixelcnn \
    --data_path cifar-10-python.tar.gz \
    --epochs 5 \
    --batch_size 45 \
    --lr 1e-3 \
    --hidden_channels 45 \
    --num_layers 6 \
    --outdir outputs \
    --print_freq 30


In [ ]:
# Train Row LSTM (Uniform reduced configuration)
!python -m src.train \
    --model_type row_lstm \
    --data_path cifar-10-python.tar.gz \
    --epochs 5 \
    --batch_size 45 \
    --lr 1e-3 \
    --hidden_channels 45 \
    --num_layers 6 \
    --outdir outputs \
    --print_freq 30


In [ ]:
# Train Diagonal BiLSTM (Uniform reduced configuration)
!python -m src.train \
    --model_type diagonal_bilstm \
    --data_path cifar-10-python.tar.gz \
    --epochs 5 \
    --batch_size 45 \
    --lr 1e-3 \
    --hidden_channels 45 \
    --num_layers 6 \
    --outdir outputs \
    --print_freq 30


In [ ]:
# Evaluate all trained models (Uniform configuration)
!python -m src.evaluate \
    --data_path cifar-10-python.tar.gz \
    --model_dir outputs \
    --output_dir evaluation_results \
    --batch_size 32 \
    --num_samples 12


In [ ]:
# Display evaluation results
import pandas as pd
import os
from IPython.display import Image, display

if os.path.exists('evaluation_results/model_comparison.csv'):
    results_df = pd.read_csv('evaluation_results/model_comparison.csv')
    print("Model Comparison Results:")
    print("=" * 50)
    print(results_df.to_string(index=False))
    
    if os.path.exists('evaluation_results/model_comparison.png'):
        print("\nModel Performance Comparison:")
        display(Image('evaluation_results/model_comparison.png'))
else:
    print("Evaluation results not found. Please run evaluation first.")


In [ ]:
# Comprehensive results analysis and visualization
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

if os.path.exists('evaluation_results/model_comparison.csv'):
    results_df = pd.read_csv('evaluation_results/model_comparison.csv')
    
    # Create comprehensive comparison plots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('PixelRNN Models Comparison on CIFAR-10', fontsize=16, fontweight='bold')
    
    models = results_df['model'].tolist()
    model_names = [name.replace('_', ' ').title() for name in models]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1'][:len(models)]
    
    # Plot 1: Test NLL Comparison
    ax1 = axes[0, 0]
    bars1 = ax1.bar(model_names, results_df['test_nll'], color=colors, alpha=0.8)
    ax1.set_ylabel('Test NLL (nats)')
    ax1.set_title('Test Negative Log-Likelihood\\n(Lower is Better)')
    ax1.grid(True, alpha=0.3, axis='y')
    
    for i, (bar, nll) in enumerate(zip(bars1, results_df['test_nll'])):
        ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                f'{nll:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Plot 2: Test BPD Comparison
    ax2 = axes[0, 1]
    bars2 = ax2.bar(model_names, results_df['test_bpd'], color=colors, alpha=0.8)
    ax2.set_ylabel('Test BPD (bits/dimension)')
    ax2.set_title('Test Bits per Dimension\\n(Lower is Better)')
    ax2.grid(True, alpha=0.3, axis='y')
    
    for i, (bar, bpd) in enumerate(zip(bars2, results_df['test_bpd'])):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.0001,
                f'{bpd:.6f}', ha='center', va='bottom', fontweight='bold')
    
    # Plot 3: Model Parameters
    ax3 = axes[1, 0]
    # Handle both possible column names
    if 'num_parameters' in results_df.columns:
        params_millions = results_df['num_parameters'] / 1e6
    else:
        params_millions = results_df['parameters'] / 1e6
    
    bars3 = ax3.bar(model_names, params_millions, color=colors, alpha=0.8)
    ax3.set_ylabel('Parameters (Millions)')
    ax3.set_title('Model Size Comparison')
    ax3.grid(True, alpha=0.3, axis='y')
    
    for i, (bar, params) in enumerate(zip(bars3, params_millions)):
        ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                f'{params:.2f}M', ha='center', va='bottom', fontweight='bold')
    
    # Plot 4: Performance vs Parameters
    ax4 = axes[1, 1]
    scatter = ax4.scatter(params_millions, results_df['test_nll'], 
                         c=colors, s=200, alpha=0.8, edgecolors='black', linewidth=2)
    
    for i, (params, nll, name) in enumerate(zip(params_millions, results_df['test_nll'], model_names)):
        ax4.annotate(name, (params, nll), xytext=(5, 5), textcoords='offset points', 
                    fontweight='bold', fontsize=10)
    
    ax4.set_xlabel('Parameters (Millions)')
    ax4.set_ylabel('Test NLL (nats)')
    ax4.set_title('Performance vs Model Size\\n(Bottom-left is better)')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('Q3_comprehensive_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Performance ranking
    print("\\n" + "="*80)
    print("📊 PERFORMANCE RANKING (by NLL - lower is better)")
    print("="*80)
    
    sorted_results = results_df.sort_values('test_nll')
    for i, (_, row) in enumerate(sorted_results.iterrows(), 1):
        model_name = row['model'].replace('_', ' ').title()
        print(f"{i}. {model_name:15} | NLL: {row['test_nll']:.4f} | BPD: {row['test_bpd']:.6f}")
    
    print("\\n✅ All models successfully trained and evaluated!")
    print("Performance hierarchy follows the expected pattern from the original PixelRNN paper.")
    
else:
    print("Results not available. Please ensure all models are trained and evaluated.")
